In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mohamedhanyyy/chest-ctscan-images

In [ ]:
import zipfile
zip_file_path = '/content/chest-ctscan-images.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')


In [ ]:
from datetime import datetime
import time

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate, Dense, GlobalAveragePooling2D, BatchNormalization, Flatten

import numpy as np
import cv2
import os
import io
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import datetime
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from time import time
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, LambdaCallback
from keras.layers import Input, Dropout, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.applications.resnet import ResNet50


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Define image size
image_size = 150

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    image = cv2.bilateralFilter(image, 2, 50, 50)
    image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
    image = cv2.resize(image, (image_size, image_size))
    return image

# Define data directories
data_dirs = {
    'train': '/content/Data/train',
    'test': '/content/Data/test',
    'valid': '/content/Data/valid'
}

# Initialize lists to store data
data = {split: {'images': [], 'labels': []} for split in data_dirs}

# Iterate over data directories
for split, directory in data_dirs.items():
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for file in tqdm(os.listdir(label_dir), desc=f'Processing {split}/{label}'):
            image_path = os.path.join(label_dir, file)
            data[split]['images'].append(preprocess_image(image_path))
            data[split]['labels'].append(label)

# Convert lists to NumPy arrays and normalize images
for split in data:
    data[split]['images'] = np.array(data[split]['images']) / 255.0
    data[split]['labels'] = np.array(data[split]['labels'])

# Extract unique labels present in the data
unique_labels = np.unique(np.concatenate([data[split]['labels'] for split in data]))

# Convert labels to one-hot encoding using dynamically extracted labels
label_to_index = {label: i for i, label in enumerate(unique_labels)}
for split in data:
    data[split]['labels'] = tf.keras.utils.to_categorical([label_to_index[label] for label in data[split]['labels']], num_classes=len(unique_labels))


# Shuffle training data
data['train']['images'], data['train']['labels'] = shuffle(data['train']['images'], data['train']['labels'], random_state=42)

# Split dataset into training, validation, and test sets
train_val_split = 0.2
val_test_split = 0.5
x_train, x_val_test, y_train, y_val_test = train_test_split(data['train']['images'], data['train']['labels'], test_size=train_val_split, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=val_test_split, random_state=42)


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Define image size
image_size = 150

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    image = cv2.bilateralFilter(image, 2, 50, 50)
    image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
    image = cv2.resize(image, (image_size, image_size))
    return image

# Define data directories
data_dirs = {
    'train': '/content/Data/train',
    'test': '/content/Data/test',
    'valid': '/content/Data/valid'
}

# Initialize lists to store data
data = {split: {'images': [], 'labels': []} for split in data_dirs}

# Iterate over data directories
for split, directory in data_dirs.items():
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for file in tqdm(os.listdir(label_dir), desc=f'Processing {split}/{label}'):
            image_path = os.path.join(label_dir, file)
            data[split]['images'].append(preprocess_image(image_path))
            data[split]['labels'].append(label)

# Convert lists to NumPy arrays and normalize images
for split in data:
    data[split]['images'] = np.array(data[split]['images']) / 255.0
    data[split]['labels'] = np.array(data[split]['labels'])

# Extract unique labels present in the data
unique_labels = np.unique(np.concatenate([data[split]['labels'] for split in data]))

# Convert labels to one-hot encoding using dynamically extracted labels
label_to_index = {label: i for i, label in enumerate(unique_labels)}
for split in data:
    data[split]['labels'] = tf.keras.utils.to_categorical([label_to_index[label] for label in data[split]['labels']], num_classes=len(unique_labels))

# Shuffle training data
data['train']['images'], data['train']['labels'] = shuffle(data['train']['images'], data['train']['labels'], random_state=42)

# Split dataset into training, validation, and test sets
train_val_split = 0.2
val_test_split = 0.5
x_train, x_val_test, y_train, y_val_test = train_test_split(data['train']['images'], data['train']['labels'], test_size=train_val_split, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=val_test_split, random_state=42)

# Apply data augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
datagen.fit(x_train)

In [ ]:
!pip install keras-tuner

In [ ]:
num_classes = len(unique_labels)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import kerastuner as kt
import numpy as np
from time import time

# Define Monte Carlo Dropout CNN model
def build_model(hp):
    inputs = layers.Input(shape=(150, 150, 3))

    # Convolutional layers
    x = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Flatten layer
    x = layers.Flatten()(x)

    # Dense layers with Monte Carlo Dropout
    for i in range(hp.Int('num_layers', min_value=1, max_value=3, step=1)):
        x = layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                         activation='relu')(x)
        x = layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1))(x)

    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 5e-4, 1e-4])),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

    return model

# Define a tuner
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='keras_tuner_dir',
                     project_name='monte_carlo_cnn')

# Search for the best hyperparameters
tuner.search(x_train, y_train,
             epochs=10,
             validation_data=(x_val, y_val))

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Fit the best model
start_time = time()
history = best_model.fit(x_train, y_train,
                         epochs=10,
                         validation_data=(x_val, y_val))
end_time = time()

# Evaluate the best model on test data using Monte Carlo Dropout
mc_dropout_model = tf.keras.models.Sequential([best_model])
mc_dropout_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                         metrics=['accuracy'])

mc_dropout_predictions = mc_dropout_model.predict(x_test, batch_size=None, verbose=1)
mc_dropout_accuracy = np.mean(np.argmax(mc_dropout_predictions, axis=-1) == np.argmax(y_test, axis=-1))

# Print metrics
print("Monte Carlo Dropout Test Accuracy:", mc_dropout_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Monte Carlo Dropout Test Acc: {mc_dropout_accuracy}')